1. **resolve_product(A)** → suy luận **category B** (và brand/sku nếu có) để thu hẹp tìm kiếm.
2. Các hàm phân tích chỉ **trả dữ liệu (không vẽ)**, luôn nhận `detected_category` (từ bước 1) + bộ lọc sàn… để chạy nhanh.
3. Mọi kết quả trả về theo chuẩn:

```json
{
  "data": [ ... list of records ... ],
  "meta": {
    "product_query": "A",
    "detected_category": "B",
    "confidence": 0.0-1.0,
    "filters": { ... platforms, time_window, brand/sku ... },
    "notes": "...",
    "ts_generated": "ISO-8601"
  }
}
```

---

# 0) Helper bắt buộc

## `resolve_product(A)`

**Mục đích:** phân loại “A” vào **category** trong danh sách chuẩn của bạn (Mobiles, Laptops, …), đồng thời cố gắng suy luận `brand`, các từ khóa tìm tương đương để lọc nhanh.

**Input**

* `A: str` — tên sản phẩm người dùng nhập (“iPhone 15 Pro Max”, “Sony WH-1000XM5”…)
* `catalog_categories: list[str]` — danh mục chuẩn của bạn
* `brand_list: list[str]` — danh sách brand chuẩn
* `hint: dict = {platforms?, brand?, category?}` — gợi ý (optional)

**Output**

```json
{
  "data": {
    "detected_category": "Mobiles",
    "brand_guess": "Apple",
    "query_tokens": ["iphone","15","pro","max"],
    "platforms": ["Lazada","Shopee","Tiki","TikTokShop","Sendo"]
  },
  "meta": {
    "product_query": "iPhone 15 Pro Max",
    "confidence": 0.92,
    "notes": "fuzzy match name→brand; regex sku; category mapping rules"
  }
}
```

> Từ đây, tất cả hàm bên dưới đều dùng `detected_category` để lọc nhanh tập dữ liệu; đồng thời có thể lọc chặt thêm theo `brand_guess` nếu bạn bật cờ.

---

# 1) Product / Market Overview (cho **sản phẩm A**)

## 1.1 `describe_price()`

**Mục đích:** thống kê **giá** của các **listing** thuộc sản phẩm A (ưu tiên cùng brand nếu có), chia theo sàn.

**Input**

* `A: str`
* `detected_category: str` (từ `resolve_product`)
* `platforms: list[str] = None`
* `use_brand_lock: bool = True` — nếu có `brand_guess` thì khóa lọc theo đó
* `min_reviews: int = 0`
* `by_platform: bool = True`

**Output**

* Nếu `by_platform=True`:

```
data: [
  { "platform": "Lazada", "count": 128,
    "min_price": 27990000, "q10": 30990000, "median_price": 32990000,
    "mean_price": 33210000, "q90": 35990000, "max_price": 39990000,
    "std_price": 2100000 }
  , ...
]
```

* Nếu `by_platform=False` → một bản ghi tổng.

---

## 1.2 `rating_distribution()`

**Mục đích:** phân phối **rating** cho các listing khớp sản phẩm A, có thể nhóm theo brand (trường hợp A có nhiều brand tương đương).

**Input**

* `A`, `detected_category`
* `platforms=None`
* `bins:int=20`
* `group_by_brand:bool=True`
* `use_brand_lock: bool=True`
* `min_reviews:int=0`

**Output**

```
data: [
  { "bucket_left": 3.5, "bucket_right": 3.6, "count": 42, "brand": "Apple" },
  ...
]
meta: {"bin_edges":[...]}
```

---

## 1.3 `sold_distribution()`

**Mục đích:** phân bố **sold** của A theo sàn.

**Input**

* `A`, `detected_category`
* `platforms=None`
* `bins: list[int] | int = [0,10,50,100,500,1000,5000,10000]`
* `use_brand_lock=True`
* `min_reviews=0`

**Output**

```
data: [
  { "platform":"Lazada","bin_left":100,"bin_right":500,"count":86,"pct":0.31 }, ...
]
```

---

## 1.4 `category_count_plot()`

**Mục đích:** đếm số **listing** của A so với các **sub-category** (nếu category của bạn có phân tầng) hoặc để benchmark vị trí A trong category.

**Input**

* `A`, `detected_category`
* `platforms=None`
* `sublevel_field:str="categories"` — cột dùng làm “nhóm con” nếu có (vd. “Mobiles > Smartphones”)
* `top_k:int=None`
* `use_brand_lock=True`

**Output**

```
data: [ { "categories": "Mobiles>Smartphones", "product_count": 412 }, ... ]
```

---

## 1.5 `brand_share()`

**Mục đích:** **thị phần brand** cho các listing tương đương sản phẩm A (trường hợp A có nhiều brand cạnh tranh, ví dụ “tai nghe chống ồn cao cấp”).

**Input**

* `A`, `detected_category`
* `platforms=None`
* `metric: 'sku' | 'revenue_est'='sku'`  (revenue_est = sold*price)
* `normalize: bool=True`
* `use_brand_lock: bool=False`  (để mở cạnh tranh chéo brand)
* `min_reviews=0`

**Output**

```
data: [
  { "platform":"Lazada", "brand":"Apple", "value": 128, "share_pct": 62.5 }, ...
]
```

---

# 2) Seller & Brands (cho **A**)

## 2.1 `top_sellers()`

**Mục đích:** top seller bán sản phẩm A (nhiều listing nhất **hoặc** tổng sold cao nhất), theo sàn.

**Input**

* `A`, `detected_category`
* `platforms=None`
* `by:'product_count'|'sold'='product_count'`
* `top_k:int=20`
* `use_brand_lock=True`
* `min_reviews=0`

**Output**

```
data: [
  { "rank":1, "seller_name":"Apple Official Store", "platform":"Lazada", "value": 37 },
  ...
]
```

## 2.2 `top_brands()`

**Mục đích:** nếu A là “dòng sản phẩm” có nhiều hãng tương tự, trả về brand có **doanh thu ước tính** hoặc **sold** cao nhất.

**Input**

* `A`, `detected_category`
* `platforms=None`
* `by:'revenue_est'|'sold'='revenue_est'`
* `top_k:int=20`
* `use_brand_lock: bool=False`  (để mở cạnh tranh nhiều hãng)

**Output**

```
data: [ { "rank":1, "brand":"Apple", "platform":"Lazada", "value": 7.23e11 }, ... ]
```

## 2.3 `seller_diversity_index()`

**Mục đích:** đo độ đa dạng **danh mục** của từng seller có bán A (hữu ích khi A thường do reseller đa ngành cung ứng).

**Input**

* `A`, `detected_category`
* `platforms=None`
* `min_products:int=5`
* `use_brand_lock=True`

**Output**

```
data: [
  { "seller_name":"TechZoneVN", "platform":"Shopee",
    "product_count": 85, "unique_categories": 7, "diversity_index": 1.86 }
]
```

---

# 3) Price / Competitiveness (cho **A**)

## 3.1 `price_range_by_category()`

**Mục đích:** dải giá bền vững của **A** trong **detected_category** (để vẽ box/interval so sánh sàn).

**Input**

* `A`, `detected_category`
* `platforms=None`
* `brand:str=None`  — ghi đè khóa
* `quantiles: (float,float)=(0.1,0.9)`
* `use_brand_lock=True`
* `min_reviews=0`

**Output**

```
data: [
  { "platform":"Lazada", "min_price": 27990000, "q_low": 30990000, "median_price": 32990000,
    "q_high": 35990000, "max_price": 39990000, "count": 128 }
  , ...
]
```

---

## (tuỳ chọn thêm)

### `roi_table_for_A()`

* **Mục đích:** hiệu suất `roi=sold/price` cho các listing của A theo sàn/brand.
* **Input:** `A`, `detected_category`, `platforms=None`, `group_by:'platform'|'seller'|'brand'='platform'`
* **Output:** `data: [{group, roi_mean, roi_median, count}]`
